## Autogen Team

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

In [2]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create an instance of the OpenAIChatCompletionClient
model_client = OpenAIChatCompletionClient(api_key=api_key, model="gpt-4o")

In [ ]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.base import TaskResult
from autogen_agentchat.messages import TextMessage


# Create the coding agent.
coding_agent = AssistantAgent(
    name="CODING_AGENT",
    model_client=model_client,
    system_message="You are a helpful AI assistant expert in the Python Programming. Generate the code for the task.",
)

# Create the testing agent.
testing_agent = AssistantAgent(
    name="TESTING_AGENT",
    model_client=model_client,
    system_message="You are a helpful AI assistant expert in the Python Programming. Test the code for the task.",
)                                                                                                           

# Create the coding editor agent.
coding_editor_agent = AssistantAgent(
    name="CODING_EDITOR_AGENT",
    model_client=model_client,
    system_message="You are a helpful AI assistant expert in the Python Programming. Edit the code for the task.",
)

# Create the group chat.
team = RoundRobinGroupChat(
    participants=[coding_agent, testing_agent, coding_editor_agent],
    max_turns=3, # Maximum number of messages/turn use by the agents combining in a team.
)

async def test_team():
    task = TextMessage(content='Write a simple Hello world code ?',source='User')
    
    result = await team.run(task=task)

    for each_agent_message in result.messages:
        print(f'{each_agent_message.source}  : {each_agent_message.content}' )
        print('\n \n')


await test_team()


User  : Write a simple Hello world code ?

 

CODING_AGENT  : Sure! Here is a simple "Hello, World!" program in Python:

```python
print("Hello, World!")
```

Simply run this code in a Python environment to see "Hello, World!" printed to the console.

 

TESTING_AGENT  : You can run the code by saving it to a file with a `.py` extension, such as `hello_world.py`, and then executing it in a terminal or command line with the command:

```bash
python hello_world.py
```

This will output:

```
Hello, World!
```

 

CODING_EDITOR_AGENT  : If you're using an interactive Python session, like a Jupyter Notebook or an IDE with an integrated console, you can simply enter:

```python
print("Hello, World!")
```

After executing, it will display:

```
Hello, World!
``` 

This is a fundamental example to demonstrate basic syntax in Python.

 



In [5]:
## Stream the result
from autogen_agentchat.base import TaskResult
await team.reset()  # Reset the team for a new task.
async for message in team.run_stream(task="Write a simple Hello world code ?"):  # type: ignore
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

source='user' models_usage=None metadata={} content='Write a simple Hello world code ?' type='TextMessage'
source='CODING_AGENT' models_usage=RequestUsage(prompt_tokens=38, completion_tokens=53) metadata={} content='Certainly! Here is a simple Python program that prints "Hello, World!" to the console:\n\n```python\nprint("Hello, World!")\n```\n\nThis line of code uses the `print()` function to output the text "Hello, World!" to the console.' type='TextMessage'
source='TESTING_AGENT' models_usage=RequestUsage(prompt_tokens=99, completion_tokens=248) metadata={} content='To test the provided code, you can follow these steps:\n\n1. **Set Up a Python Environment:**\n   - Make sure Python is installed on your system. You can download it from [python.org](https://www.python.org/downloads/) if it\'s not already installed.\n\n2. **Create a Python File:**\n   - Open a text editor (like Notepad, Sublime Text, VSCode, or PyCharm) and create a new file.\n   - Copy and paste the code into this file

In [6]:
from autogen_agentchat.ui import Console

await team.reset()  # Reset the team for a new task.
await Console(team.run_stream(task="Write a simple Hello world code ?"))  # Stream the messages to the console.


---------- TextMessage (user) ----------
Write a simple Hello world code ?
---------- TextMessage (CODING_AGENT) ----------
Certainly! Here's a simple "Hello, World!" program in Python:

```python
# This is a simple Python program that prints "Hello, World!" to the console.
def main():
    print("Hello, World!")

if __name__ == "__main__":
    main()
```

You can run this code in any Python environment to see the output.
---------- TextMessage (TESTING_AGENT) ----------
To run this simple "Hello, World!" program, save the code in a file with a `.py` extension, for example `hello_world.py`, and then execute it using a Python interpreter. Open a terminal or command prompt, navigate to the directory where your file is saved, and run the following command:

```bash
python hello_world.py
```

You should see the output:

```
Hello, World!
```

This confirms that your "Hello, World!" program is working correctly.
---------- TextMessage (CODING_EDITOR_AGENT) ----------
If you're using Python 3

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a simple Hello world code ?', type='TextMessage'), TextMessage(source='CODING_AGENT', models_usage=RequestUsage(prompt_tokens=38, completion_tokens=72), metadata={}, content='Certainly! Here\'s a simple "Hello, World!" program in Python:\n\n```python\n# This is a simple Python program that prints "Hello, World!" to the console.\ndef main():\n    print("Hello, World!")\n\nif __name__ == "__main__":\n    main()\n```\n\nYou can run this code in any Python environment to see the output.', type='TextMessage'), TextMessage(source='TESTING_AGENT', models_usage=RequestUsage(prompt_tokens=118, completion_tokens=101), metadata={}, content='To run this simple "Hello, World!" program, save the code in a file with a `.py` extension, for example `hello_world.py`, and then execute it using a Python interpreter. Open a terminal or command prompt, navigate to the directory where your file is saved, and run th